In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 64
states = ["WA"]
raceDB = build_dataset('gru_inputs_new_simple_new.npy', hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']
(1202877, 22)
(101505, 24)
Latest date = 2023-04-09 00:00:00
num_features_per_dog=20


  0%|          | 0/3540 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/14234 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 14234, number of unique dogs = 3541
0        (Cannington, 275.0)
1        (Cannington, 520.0)
2        (Cannington, 380.0)
3        (Cannington, 520.0)
4        (Cannington, 275.0)
                ...         
14229    (Cannington, 275.0)
14230    (Cannington, 275.0)
14231    (Cannington, 520.0)
14232    (Cannington, 380.0)
14233    (Cannington, 380.0)
Length: 14234, dtype: object


In [7]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 11259, Test examples 2975


In [8]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(365), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 11, 30)
start_date=datetime.date(2020, 11, 30)
period=datetime.date(2021, 11, 30)
start_date=datetime.date(2021, 11, 30)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 7, 31))]


In [9]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-11-30
2021-11-30
2022-07-31
Train examples [4125, 4357, 2764]


In [10]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    print(bi)
    print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in raceDB.dog_ids:
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in dog.races.keys() if x in batch]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

0
['500539179', '500539180', '500539181', '500539182', '500539188', '500539183', '500539187', '500539189', '500539184', '500539190', '500539185', '500539186', '501809130', '500541205', '500543223', '500543224', '500541210', '500541209', '500543225', '500541204', '500541203', '500541208', '500541202', '500541206', '500541207', '501820139', '501820142', '501820149', '501818146', '501820141', '501818145', '501820140', '501820150', '501820143', '501820148', '501820145', '501820144', '501820147', '501820146', '502577739', '502577746', '502577738', '502577744', '502577748', '502577745', '502577742', '502577747', '502577743', '502577740', '502577741', '502271138', '502271140', '502271141', '502271139', '502271132', '502271142', '502271137', '502271135', '502271133', '502271136', '502271134', '502577795', '502577789', '502577800', '502577793', '502577791', '502577797', '502577799', '502577784', '502577782', '502577786', '502577798', '502577780', '504089699', '504089689', '504089698', '50408969

In [11]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [12]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [1666, 1709, 1389]
Train examples [1666, 1709, 1389]
Train examples [4125, 4357, 2764]
Train examples [4125, 4357, 2764]


In [13]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/3540 [00:00<?, ?it/s]

In [14]:
raceDB.attach_races_to_dog_inputv2()

  0%|          | 0/3541 [00:00<?, ?it/s]

In [15]:
batchnorm = torch.nn.BatchNorm1d(84).to('cuda:0')

In [16]:
def batch_norm_pack_seq(seq):
    x,idx = pad_packed_sequence(seq)
    print(x.shape)
    x = x.transpose(0,1).transpose(1,2)
    x = batchnorm(x.float())
    x = x.transpose(2,1).transpose(1,0)
    x = unpad_sequence(x, idx)
    x = pack_sequence(x, enforce_sorted=False)
    return x

In [17]:
# test_idx = range(0,len(raceDB.test_dog_ids))
# # packed_x = [batch_norm_pack_seq(pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False)).to('cuda:0') for x in train_dog_input]
# packed_y = batch_norm_pack_seq(pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False))

In [18]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = [pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_x_basic = [pack_sequence([torch.stack(n,0) for n in [[z.stats for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
packed_y_basic = pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [19]:
train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [20]:
[sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

[17.721488595438174, 18.334698654183732, 14.078473722102231]

In [21]:
unpack_sequence(packed_x[0])[0][2]

tensor([ 3.0200e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [22]:
raceDB.batches = {'num_batches':len(train_dogs),'dogs':train_dogs,'train_dog_input':train_dog_input,'batch_races':batch_races,'batch_races_ids':batch_races_ids,'packed_x':packed_x,'packed_x_basic':packed_x_basic,'packed_y_basic':packed_y_basic, 'packed_y':packed_y}

In [23]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)
      if config.input_type == 'full':
        print('here1')
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]
      elif config.input_type == 'basic':
        print('here')
        raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
        raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]

      print(f"{input_size=}")


      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv2(input_size,config['hidden_size'], num_layers=config['num_layers'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      # criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      criterion = nn.CrossEntropyLoss(reduction='none')
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      # optimizer = optim.Adadelta(model.parameters())
      # optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [24]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'stats':raceDB.stats_cols,'input_type':'basic','num_layers':3,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.001, 'loss': 'L1', 'l1_beta':0.1,   'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [25]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [26]:
wandb_config_static

{'hidden_size': 64,
 'stats': ['dist_last__1',
  'box_last__1',
  'speed_avg_1',
  'split_speed_avg_1',
  'split_margin_avg_1',
  'margin_avg_1',
  'first_out_avg_1',
  'post_change_avg_1',
  'races_1',
  'wins_1',
  'weight_',
  'min_time_',
  'min_split_time_',
  'last_start_price'],
 'input_type': 'basic',
 'num_layers': 3,
 'batch_size': 750,
 'dropout': 0.3,
 'epochs': 10000,
 'f1_layer_size': 256,
 'f2_layer_size': 64,
 'learning_rate': 0.001,
 'loss': 'L1',
 'l1_beta': 0.1,
 'optimizer': 'adamW',
 'validation_split': 0.1,
 'batch_before_backwards': 7}

In [27]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [28]:
torch.cuda.empty_cache()

In [29]:
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

In [ ]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [44]:
race = raceDB.batches['batch_races'][0][2]
[d for d in race.dogs]

In [26]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_size":{"value":69},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['full','basic']},
    "num_layers": {"values": [2,3,4]},
    'hidden_size':{'values':[32,64,128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    # "hidden_size": {"value":hidden_size}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [500]},
        "num_layers": {"values": [2,3,4]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.002,
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_size': {'value': 69},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [500]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'values': [32, 64, 128]},
                'input_type': {'values': ['full', 'basic']},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.002,
                                  'min': 1e-05},
                'len_data': {'value': 14234},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2, 3, 4]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'batch_size': {'value': 69},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'input_type': {'values': ['full', 'basic']},
  'num_layers': {'values': [2, 3, 4]},
  'hidden_size': {'values': [32, 64, 128]},
  'len_data': {'value': 14234},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.002}}}

In [27]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 457uc5ro
Sweep URL: https://wandb.ai/nickojelly/NEW%20GRU-sweeps/sweeps/457uc5ro


wandb: Agent Starting Run: ev796qhm with config:
wandb: 	batch_size: 69
wandb: 	dropout: 0.3
wandb: 	epochs: 500
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 32
wandb: 	input_type: basic
wandb: 	learning_rate: 0.00011566882907696976
wandb: 	len_data: 14234
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 69, 'dropout': 0.3, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'input_type': 'basic', 'learning_rate': 0.00011566882907696976, 'len_data': 14234, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
500
{'batch_size': 69, 'dropout': 0.3, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'input_type': 'basic', 'learning_rate': 0.00011566882907696976, 'len_data': 14234, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
basic
here
input_size=20
GRUNetv2(
  (gru): GRU(20, 32, num_layers=2, dropout=0.3)
  (batchnorm): BatchNorm1d(326, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=32, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=326, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(i

  0%|          | 0/500 [00:00<?, ?it/s]

...

ROI < 30,▅▁▂▃▇█▅▆▃▂▄▄▆▄▆▄▄▅▆▆▇▆▄▅▃
ROI < 30 2,▅▁▃▄▇█▅▄▃▁▂▃▄▃▅▄▅▅▆▆▆▆▄▄▄
accuracy,▁▄▅▆███████████████████▇▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
flat_simple,▃▁▄▇▅█▆▂▄▄▅▄▅▄▅▇▇▆▅▆▅▆▅▃▄
loss_1,█▇▇▇▆▆▅▅▅▅▅▅▅▄▄▄▅▄▄▄▃▄▄▃▃▄▄▃▃▂▃▃▂▂▁▃▂▂▁▁
loss_val,█▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂
multibet outlay < 30,▃▁▂▁▁▁▁▂▂▂▂▃▃▄▅▅▅▅▆▇▇████
multibet profit,▁▂▄▅██▇▇▆▄▅▆▆▅▅▄▄▆▆▆▇▆▇▅▆
multibet profit < 30,▅▃▃▅▇█▆▆▄▃▄▅▆▄▅▃▄▄▄▅▅▅▂▃▁
multibet profit < 30 sd,▆▁▃▂▂▂▁▂▁▁▂▃▃▃▄▃▄▄▆▇▇█▇█▇


wandb: Agent Starting Run: omfakt7h with config:
wandb: 	batch_size: 69
wandb: 	dropout: 0.3
wandb: 	epochs: 500
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 128
wandb: 	input_type: full
wandb: 	learning_rate: 0.0012794930188558431
wandb: 	len_data: 14234
wandb: 	loss: CEL
wandb: 	num_layers: 3
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 69, 'dropout': 0.3, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 128, 'input_type': 'full', 'learning_rate': 0.0012794930188558431, 'len_data': 14234, 'loss': 'CEL', 'num_layers': 3, 'optimizer': 'adamW', 'validation_split': 0.1}
500
{'batch_size': 69, 'dropout': 0.3, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 128, 'input_type': 'full', 'learning_rate': 0.0012794930188558431, 'len_data': 14234, 'loss': 'CEL', 'num_layers': 3, 'optimizer': 'adamW', 'validation_split': 0.1}
full
here1
input_size=20
GRUNetv2(
  (gru): GRU(20, 128, num_layers=3, dropout=0.3)
  (batchnorm): BatchNorm1d(1094, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=128, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear

  0%|          | 0/500 [00:00<?, ?it/s]

created path
